In [ ]:
import numpy as np
import qubex as qx

from qubex.simulator import (
    Control,
    Coupling,
    QuantumSimulator,
    QuantumSystem,
    SimulationResult,
    Transmon,
)

qx.pulse.set_sampling_period(0.1)

In [ ]:
# units: GHz, ns
transmon_dimension = 3

control_qubit_label = "Q04"
control_qubit_frequency = 7157.231e-3
control_anharmonicity = -393.715e-3

target_qubit_label = "Q01"
target_qubit_frequency = 7532.295e-3
target_anharmonicity = -487.412e-3

relaxation_rate = 0.05e-3
dephasing_rate = 0.05e-3

coupling_strength = 5e-3

frequency_diff = control_qubit_frequency - target_qubit_frequency

control_drive_frequency = control_qubit_frequency
target_drive_frequency = target_qubit_frequency

drag_duration = 24

cr_amplitude = 0.75 * abs(frequency_diff)
cr_phase = 0.25 * np.pi
cr_ramptime = 16
cr_beta = -1 / (2 * np.pi * frequency_diff)

xt_ratio = 0.01
xt_amplitude = cr_amplitude * xt_ratio
xt_phase = 0.5 * np.pi

cancel_beta = -1 / (2 * np.pi * target_anharmonicity)

skew = 32

In [ ]:
control_qubit = Transmon(
    label=control_qubit_label,
    dimension=transmon_dimension,
    frequency=control_qubit_frequency,
    anharmonicity=control_anharmonicity,
    relaxation_rate=relaxation_rate,
    dephasing_rate=dephasing_rate,
)

target_qubit = Transmon(
    label=target_qubit_label,
    dimension=transmon_dimension,
    frequency=target_qubit_frequency,
    anharmonicity=target_anharmonicity,
    relaxation_rate=relaxation_rate,
    dephasing_rate=dephasing_rate,
)

couplings = [
    Coupling(
        pair=(control_qubit_label, target_qubit_label),
        strength=coupling_strength,
    ),
]

system = QuantumSystem(
    objects=[control_qubit, target_qubit],
    couplings=couplings,
)

simulator = QuantumSimulator(system)

In [ ]:
def drag_pi_pulse(
    duration: float,
    anharmonicity: float,
) -> qx.Pulse:
    pulse = qx.pulse.Drag(
        duration=duration,
        amplitude=1,
        beta=-0.5 / (2 * np.pi * anharmonicity),
    )
    norm_factor = np.pi / float(np.sum(np.abs(pulse.values) * pulse.SAMPLING_PERIOD))
    pulse = pulse.scaled(norm_factor)
    return pulse


pi_pulse = drag_pi_pulse(
    duration=drag_duration,
    anharmonicity=control_qubit.anharmonicity,
)
pi_pulse.plot()

result = simulator.mesolve(
    controls=[
        Control(
            target=control_qubit_label,
            frequency=control_drive_frequency,
            waveform=pi_pulse,
        )
    ],
)
result.display_bloch_sphere(control_qubit_label)
result.show_last_population(control_qubit_label)

In [ ]:
def cr_sequence(
    control_drive_frequency: float,
    target_drive_frequency: float,
    duration: float,
    ramptime: float,
    cr_amplitude: float,
    cr_phase: float,
    cr_beta: float = 0.0,
    xt_amplitude: float = 0.0,
    xt_phase: float = 0.0,
    cancel_amplitude: float = 0.0,
    cancel_phase: float = 0.0,
    cancel_beta: float = 0.0,
    skew: float = 0.0,
    echo: bool = False,
):
    cr_waveform = qx.pulse.FlatTop(
        duration=duration,
        amplitude=2 * np.pi * cr_amplitude,
        tau=ramptime,
        phase=cr_phase,
        beta=cr_beta,
    )
    cancel_waveform = qx.pulse.FlatTop(
        duration=duration,
        amplitude=2 * np.pi * cancel_amplitude,
        tau=ramptime,
        phase=cancel_phase,
        beta=cancel_beta,
    )
    crosstalk_waveform = qx.pulse.FlatTop(
        duration=duration,
        amplitude=2 * np.pi * xt_amplitude,
        tau=ramptime,
        phase=xt_phase,
        beta=cr_beta,
    )
    targets = [
                qx.PulseChannel(
                    label="Control",
                    frequency=control_drive_frequency,
                    target=control_qubit_label,
                ),
                qx.PulseChannel(
                    label="CR",
                    frequency=target_drive_frequency,
                    target=control_qubit_label,
                ),
                qx.PulseChannel(
                    label="Crosstalk",
                    frequency=target_drive_frequency,
                    target=target_qubit_label,
                ),
                qx.PulseChannel(
                    label="Target",
                    frequency=target_drive_frequency,
                    target=target_qubit_label,
                ),
            ]
    with qx.PulseSchedule(targets) as cr:
        cr.add("CR", cr_waveform)
        cr.add("Crosstalk", crosstalk_waveform)
        cr.add("Target", cancel_waveform)

    if not echo:
        sched = cr
    else:
        with qx.PulseSchedule(targets) as ecr:
            ecr.call(cr)
            ecr.barrier()
            ecr.add("Control", pi_pulse)
            ecr.barrier()
            ecr.call(cr.scaled(-1))
            ecr.barrier()
            ecr.add("Control", pi_pulse)
        sched = ecr

    if skew == 0:
        return sched
    else:
        seqs = sched.get_sequences()
        with qx.PulseSchedule(targets) as skewed:
            if skew > 0:
                skewed.add("Control", seqs["Control"])
                skewed.add("CR", seqs["CR"])
                skewed.add("Crosstalk", seqs["Crosstalk"])
                skewed.add("Target", qx.PulseArray([qx.Blank(skew), seqs["Target"]]))
            else:
                skewed.add("Control", qx.PulseArray([qx.Blank(-skew), seqs["Control"]]))
                skewed.add("CR", qx.PulseArray([qx.Blank(-skew), seqs["CR"]]))
                skewed.add("Crosstalk", qx.PulseArray([qx.Blank(-skew), seqs["Crosstalk"]]))
                skewed.add("Target", seqs["Target"])
        return skewed

In [ ]:
duration = 200
cancel_amplitude = 0.01
cancel_phase = 0.0

cr_sequence(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    duration=duration,
    ramptime=cr_ramptime,
    cr_amplitude=cr_amplitude,
    cr_phase=cr_phase,
    cr_beta=cr_beta,
    xt_amplitude=xt_amplitude,
    xt_phase=xt_phase,
    cancel_amplitude=cancel_amplitude,
    cancel_phase=cancel_phase,
    cancel_beta=cancel_beta,
    skew=skew,
    echo=True,
).plot(
    title="Echoed Cross Resonance Sequence",
    divide_by_two_pi=True,
)

In [ ]:
def simulate_cr(
    control_drive_frequency: float,
    target_drive_frequency: float,
    cr_amplitude: float,
    cr_phase: float,
    cr_beta: float = 0.0,
    xt_amplitude: float = 0.0,
    xt_phase: float = 0.0,
    cancel_amplitude: float = 0.0,
    cancel_phase: float = 0.0,
    cancel_beta: float = 0.0,
    duration: float = 1000,
    ramptime: float = 0.0,
    skew: float = 0.0,
    echo: bool = False,
    control_state: str = "0",
    n_samples: int = 256,
    plot: bool = False,
) -> SimulationResult:
    controls = cr_sequence(
        control_drive_frequency=control_drive_frequency,
        target_drive_frequency=target_drive_frequency,
        duration=duration,
        ramptime=ramptime,
        cr_amplitude=cr_amplitude,
        cr_phase=cr_phase,
        cr_beta=cr_beta,
        xt_amplitude=xt_amplitude,
        xt_phase=xt_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        cancel_beta=cancel_beta,
        skew=skew,
        echo=echo,
    )
    initial_state = system.state(
        {
            control_qubit.label: control_state,
            target_qubit.label: "0",
        },
    )
    result = simulator.mesolve(
        controls=controls,
        initial_state=initial_state,
        n_samples=n_samples,
    )
    if plot:
        # controls.plot(divide_by_two_pi=True)
        print(f"Control qubit: {control_qubit_label}")
        # result.show_last_population(control_qubit_label)
        # result.plot_population_dynamics(control_qubit_label)
        result.plot_bloch_vectors(control_qubit_label)
        result.display_bloch_sphere(control_qubit_label)
        print(f"Target qubit: {target_qubit_label}")
        result.plot_bloch_vectors(target_qubit_label)
        result.display_bloch_sphere(target_qubit_label)
    return result

In [ ]:
result_0 = simulate_cr(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude,
    cr_phase=cr_phase,
    cr_beta=cr_beta,
    xt_amplitude=xt_amplitude,
    xt_phase=xt_phase,
    duration=100,
    ramptime=cr_ramptime,
    control_state="0",
    plot=True,
)

In [ ]:
result_1 = simulate_cr(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude,
    cr_phase=cr_phase,
    cr_beta=cr_beta,
    xt_amplitude=xt_amplitude,
    xt_phase=xt_phase,
    duration=100,
    ramptime=cr_ramptime,
    control_state="1",
    plot=True,
)

In [ ]:
def hamiltonian_tomography(
    control_drive_frequency: float,
    target_drive_frequency: float,
    cr_amplitude: float,
    cr_phase: float,
    cr_beta: float = 0.0,
    xt_amplitude: float = 0.0,
    xt_phase: float = 0.0,
    cancel_amplitude: float = 0.0,
    cancel_phase: float = 0.0,
    cancel_beta: float = 0.0,
    duration: float = 1000,
    ramptime: float = 24.0,
    n_samples: int = 100,
    plot: bool = False,
) -> dict:
    result_0 = simulate_cr(
        control_drive_frequency=control_drive_frequency,
        target_drive_frequency=target_drive_frequency,
        cr_amplitude=cr_amplitude,
        cr_phase=cr_phase,
        cr_beta=cr_beta,
        xt_amplitude=xt_amplitude,
        xt_phase=xt_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        cancel_beta=cancel_beta,
        duration=duration,
        ramptime=ramptime,
        control_state="0",
        n_samples=n_samples,
        plot=plot,
    )
    result_1 = simulate_cr(
        control_drive_frequency=control_drive_frequency,
        target_drive_frequency=target_drive_frequency,
        cr_amplitude=cr_amplitude,
        cr_phase=cr_phase,
        cr_beta=cr_beta,
        xt_amplitude=xt_amplitude,
        xt_phase=xt_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        cancel_beta=cancel_beta,
        duration=duration,
        ramptime=ramptime,
        control_state="1",
        n_samples=n_samples,
        plot=plot,
    )

    times = result_0.times
    vectors_0 = result_0.get_bloch_vectors(target_qubit.label)
    vectors_1 = result_1.get_bloch_vectors(target_qubit.label)

    indices = (times >= ramptime) & (times < times[-1] - ramptime)
    times_ = times[indices] - ramptime * 0.5
    vectors_0_ = vectors_0[indices]
    vectors_1_ = vectors_1[indices]

    fit_0 = qx.fit.fit_rotation(
        times_,
        vectors_0_,
        plot=plot,
    )
    fit_1 = qx.fit.fit_rotation(
        times_,
        vectors_1_,
        plot=plot,
    )
    Omega_0 = fit_0["Omega"]
    Omega_1 = fit_1["Omega"]
    Omega = np.concatenate(
        [
            0.5 * (Omega_0 + Omega_1),
            0.5 * (Omega_0 - Omega_1),
        ]
    )
    coeffs = dict(
        zip(
            ["IX", "IY", "IZ", "ZX", "ZY", "ZZ"],
            Omega / (2 * np.pi),
        )
    )

    print("Rotation coefficients:")
    for key, value in coeffs.items():
        print(f"  {key}: {value * 1e3:+.3f} MHz")

    xt_rotation = coeffs["IX"] + 1j * coeffs["IY"]
    print()
    print("XT (crosstalk) rotation:")
    print(f"  rate  : {np.abs(xt_rotation) * 1e3:.3f} MHz")
    print(f"  phase : {np.angle(xt_rotation):.3f} rad")

    cr_rotation = coeffs["ZX"] + 1j * coeffs["ZY"]
    print()
    print("CR (cross-resonance) rotation:")
    print(f"  rate  : {np.abs(cr_rotation) * 1e3:.3f} MHz")
    print(f"  phase : {np.angle(cr_rotation):.3f} rad")

    zx90_duration = 1 / (4 * np.abs(cr_rotation))
    print()
    print("Estimated ZX90 gate:")
    print(f"  drive    : {cr_amplitude * 1e3:.1f} MHz")
    print(f"  duration : {zx90_duration:.1f} ns")
    print()

    return {
        "Omega": Omega,
        "coeffs": coeffs,
        "xt_rotation": xt_rotation,
        "cr_rotation": cr_rotation,
        "zx90_duration": zx90_duration,
    }

In [ ]:
tomography_1 = hamiltonian_tomography(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude,
    cr_phase=cr_phase,
    cr_beta=cr_beta,
    xt_amplitude=xt_amplitude,
    xt_phase=xt_phase,
    cancel_amplitude=0.0,
    cancel_phase=0.0,
    cancel_beta=0.0,
    plot=True,
)

tomography_1

In [ ]:
cr_phase_est = np.angle(tomography_1["cr_rotation"])
cr_phase_est

In [ ]:
tomography_2 = hamiltonian_tomography(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude,
    cr_phase=cr_phase - cr_phase_est,
    xt_amplitude=xt_amplitude,
    xt_phase=xt_phase,
    cancel_amplitude=0,
    cancel_phase=0,
    plot=True,
)

tomography_2

In [ ]:
cancel_pulse = -tomography_2["xt_rotation"]
cancel_amplitude = np.abs(cancel_pulse)
cancel_phase = np.angle(cancel_pulse)

cancel_amplitude, cancel_phase

In [ ]:
tomography_3 = hamiltonian_tomography(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude,
    cr_phase=cr_phase - cr_phase_est,
    xt_amplitude=xt_amplitude,
    xt_phase=xt_phase,
    cancel_amplitude=cancel_amplitude,
    cancel_phase=cancel_phase,
    plot=True,
)

tomography_3

In [ ]:
cancel_multiple = 10
cancel_pulse = (
    -tomography_2["xt_rotation"] + tomography_3["coeffs"]["ZX"] * cancel_multiple
)
cancel_amplitude = np.abs(cancel_pulse)
cancel_phase = np.angle(cancel_pulse)

cancel_amplitude, cancel_phase

In [ ]:
tomography_4 = hamiltonian_tomography(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude,
    cr_phase=cr_phase - cr_phase_est,
    xt_amplitude=xt_amplitude,
    xt_phase=xt_phase,
    cancel_amplitude=cancel_amplitude,
    cancel_phase=cancel_phase,
    plot=True,
)

tomography_4

In [ ]:
cr_duration = ((tomography_3["zx90_duration"] / 2) // 16 + 1) * 16
cr_duration

In [ ]:
cr_sequence(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    duration=cr_duration,
    ramptime=cr_ramptime,
    cr_amplitude=cr_amplitude,
    cr_phase=cr_phase - cr_phase_est,
    cr_beta=cr_beta,
    xt_amplitude=xt_amplitude,
    xt_phase=xt_phase,
    cancel_amplitude=cancel_amplitude,
    cancel_phase=cancel_phase,
    cancel_beta=cancel_beta,
    echo=True,
).plot(
    title="Echoed Cross Resonance Sequence",
    width=800,
    divide_by_two_pi=True,
)

In [ ]:
def measure(
    control_drive_frequency: float,
    target_drive_frequency: float,
    duration: float,
    ramptime: float,
    cr_amplitude: float,
    cr_phase: float,
    cr_beta: float,
    cancel_amplitude: float,
    cancel_phase: float,
    cancel_beta: float,
    control_state: str,
):
    result= simulate_cr(
        control_drive_frequency=control_drive_frequency,
        target_drive_frequency=target_drive_frequency,
        cr_amplitude=cr_amplitude,
        cr_phase=cr_phase,
        cr_beta=cr_beta,
        xt_amplitude=xt_amplitude,
        xt_phase=xt_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        cancel_beta=cancel_beta,
        duration=duration,
        ramptime=ramptime,
        echo=True,
        control_state=control_state,
        n_samples=2,
        plot=False,
    )
    state = result.get_bloch_vectors(target_qubit.label)[-1]
    return state[2]

In [ ]:
amplitude_min = 0
amplitude_max = abs(frequency_diff)
amplitude_range = np.linspace(amplitude_min, amplitude_max, 20)

z_values = []
for amplitude in amplitude_range:
    result = measure(
        control_drive_frequency=control_drive_frequency,
        target_drive_frequency=target_drive_frequency,
        cr_amplitude=amplitude,
        cr_phase=cr_phase - cr_phase_est,
        cr_beta=cr_beta,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        cancel_beta=cancel_beta,
        duration=cr_duration,
        ramptime=cr_ramptime,
        control_state="0",
    )
    z_values.append(result)

z_values = np.array(z_values)

qx.viz.plot(x=amplitude_range, y=z_values)

In [ ]:
fit_result = qx.fit.fit_polynomial(x=amplitude_range, y=z_values, degree=3)

cr_amplitude_calib = fit_result["root"]
cr_amplitude_calib

In [ ]:
measure(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_qubit_frequency,
    duration=cr_duration,
    ramptime=cr_ramptime,
    cr_amplitude=cr_amplitude_calib,
    cr_phase=cr_phase - cr_phase_est,
    cr_beta=cr_beta,
    cancel_amplitude=cancel_amplitude,
    cancel_phase=cancel_phase,
    cancel_beta=cancel_beta,
    control_state="0",
)

In [ ]:
result_ecr_0 = simulate_cr(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude_calib,
    cr_phase=cr_phase - cr_phase_est,
    cr_beta=cr_beta,
    xt_amplitude=xt_amplitude,
    xt_phase=xt_phase,
    cancel_amplitude=cancel_amplitude,
    cancel_phase=cancel_phase,
    cancel_beta=cancel_beta,
    duration=cr_duration,
    ramptime=cr_ramptime,
    control_state="0",
    echo=True,
    plot=True,
)

In [ ]:
result_ecr_1 = simulate_cr(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude_calib,
    cr_phase=cr_phase - cr_phase_est,
    cr_beta=cr_beta,
    xt_amplitude=xt_amplitude,
    xt_phase=xt_phase,
    cancel_amplitude=cancel_amplitude,
    cancel_phase=cancel_phase,
    cancel_beta=cancel_beta,
    duration=cr_duration,
    ramptime=cr_ramptime,
    control_state="1",
    echo=True,
    plot=True,
)

In [ ]:
print("CR parameters:")
print(f"  duration:         {cr_duration} ns")
print(f"  ramptime:         {cr_ramptime} ns")
print(f"  cr_amplitude:     {cr_amplitude * 1e3:.1f} MHz")
print(f"  cr_phase:         {cr_phase:.3f} rad")
print(f"  cancel_amplitude: {cancel_amplitude * 1e3:.1f} MHz")
print(f"  cancel_phase:     {cancel_phase:.3f} rad")